# Forward Propagation
Using matrix-vector multiplication. <br>
$\vec{x} \leftarrow$ input-layer / vector <br>
$L_{n} \leftarrow$ hidden layer $n$ / matrix <br>
$\vec{y}=L_{2} \cdot L_{1} \cdot \vec{x} \leftarrow$ output-layer / vector

In [7]:
import json
import numpy as np
import pprint

with open(r"example-2layer.json", "r") as read_file:
    example = json.load(read_file)

input_layer = list()
for a in range(int(example.get("layer1").get("size_in"))):
    input_layer.append(1)

input_layer = np.array(input_layer)

[0.62, 0.35999999999999993]


## Input and hidden layers
$\vec{x} = \begin{bmatrix} 1\\1\\1\\1\\1 \end{bmatrix}$ <br>
$L_{1} = \begin{bmatrix} 
0.5&0.2&0&0&-0.2\\
0.2&-0.5&-0.1&0.9&-0.8\\
0&0.2&0&0.1&-0.1\\
0.1&0.8&0.3&0&-0.7\end{bmatrix}, L_{2} = \begin{bmatrix} 0.5&0.2&-0.1&0.9\\ 0.2&-0.5&0.3&0.1 \end{bmatrix}$
<br>

In [ ]:
def generate_weight_matrix(layer_string):
    weights = list()
    for size in range(int(example.get(layer_string).get("size_in"))):
        weights.append(list('0' * int(example.get(layer_string).get("size_out"))))

    for weight in example.get(layer_string).get("weights"):
        for w in example.get(layer_string).get("weights").get(weight):
            weights[int(weight) - 1][int(w) - 1] = example.get(layer_string).get("weights").get(weight).get(w)

    weights = np.array(weights)
    transposed_weights = np.transpose(weights)
    return transposed_weights


# Weight matrix layer 1:
matrix_layer1 = generate_weight_matrix("layer1")
# Weight matrix layer 2:
matrix_layer2 = generate_weight_matrix("layer2")



## Forward propagation using one layer
Where the matrix is $n \times m$ and the vector is $m \times 1$<br>
$L_{1} \cdot \vec{x} = \vec{y}$ <br>
$\vec{y} = \begin{bmatrix} 
0.5&0.2&0&0&-0.2\\
0.2&-0.5&-0.1&0.9&-0.8\\
0&0.2&0&0.1&-0.1\\
0.1&0.8&0.3&0&-0.7\end{bmatrix} \cdot \begin{bmatrix} 1\\1\\1\\1\\1 \end{bmatrix}$
>$= \begin{bmatrix} 1 \cdot 0.5 + 1 \cdot 0.2 + 1 \cdot 0  + 1 \cdot 0 + 1 \cdot -0.2\\
   1 \cdot 0.2 + 1 \cdot -0.5 + 1 \cdot -0.1 + 1 \cdot 0.9 + 1 \cdot -0.8\\
   1 \cdot 0 + 1 \cdot 0.2 + 1 \cdot 0 + 1 \cdot 0.1 + 1 \cdot -0.1\\
   1 \cdot 0.1 + 1 \cdot 0.8 + 1 \cdot 0.3 + 1 \cdot 0 + 1 \cdot -0.7\end{bmatrix}$
   
$\vec{y} = \begin{bmatrix} 0.5\\-0.3\\0.2\\0.5 \end{bmatrix}$

In [ ]:
def matrix_vector_product(activation, weights):
    output_layer = list()
    for w in weights:
        vector = 0.0
        for a in range(len(activation)):
            vector1 = float(float(activation[a]) * float(w[a]))
            vector += vector1
        output_layer.append(vector)
    return output_layer



In [ ]:
# For one layer:
output_layer_1 = matrix_vector_product(input_layer, matrix_layer1)
pprint.pprint(output_layer_1)


## Forward propagation using two layers
There are atleast two methods for calculating forward propagation with multiple hidden layers. <br>
1. Use the output vector from layer n to active layer n+1 (matrix-vector-product).
2. Turn the layers into one layer using matrix multiplication (matrix-matrix-product).

### Method 1:
--
Using the output vector from ['Forward propagation using one layer'](#Forward-propagation-using-one-layer) as $\vec{x}$
and $L_{2}$ for the matrix. <br>
$\vec{x} = \begin{bmatrix} 0.5\\-0.3\\0.2\\0.5 \end{bmatrix}$<br>
$L_{2} \cdot \vec{x} = \vec{y}$ <br>
$\vec{y} = \begin{bmatrix} 0.5&0.2&-0.1&0.9\\ 0.2&-0.5&0.3&0.1 \end{bmatrix} \cdot 
\begin{bmatrix} 0.5\\-0.3\\0.2\\0.5 \end{bmatrix}$ 
>$= \begin{bmatrix}0.5 \cdot 0.5 + 0.2 \cdot -0.3 + -0.1 \cdot 0.2  + 0.9 \cdot 0.5\\
0.2 \cdot 0.5 + -0.5 \cdot -0.3 + 0.3 \cdot 0.2 + 0.1 \cdot 0.5
\end{bmatrix}$

$\vec{y} = \begin{bmatrix} 0.62\\0.36 \end{bmatrix}$ <br>
Due to floating point inaccuracy, the outcome of the Python script will probably differ by a small amount. This is also
true of the 2nd method.

In [ ]:
# When layer 1's output is used as input vector:
pprint.pprint(matrix_vector_product(np.transpose(output_layer_1), matrix_layer2))

### Method 2:
This method multiplies $L_{2}$ with $L_{1}$ and the result (matrix) with $\vec{x}$.<br>
Matrix-matrix product requires matrix A to have the dimensions $m\times n$, matrix B $n\times p$ and the resulting 
matrix AB $m\times p$. <br>
Matrix-matrix product can, thankfully (saves on code), be turned into a couple of matrix-vector products. <br>
$L_{1} = \begin{bmatrix} 
0.5&0.2&0&0&-0.2\\
0.2&-0.5&-0.1&0.9&-0.8\\
0&0.2&0&0.1&-0.1\\
0.1&0.8&0.3&0&-0.7\end{bmatrix} = \begin{bmatrix} 
 \begin{bmatrix} 0.5\\0.2\\0\\0.1 \end{bmatrix}
 \begin{bmatrix} 0.2\\-0.5\\0.2\\0.8 \end{bmatrix} 
 \begin{bmatrix} 0\\-0.1\\0\\0.3 \end{bmatrix}
 \begin{bmatrix} 0\\0.9\\0.1\\0 \end{bmatrix} 
 \begin{bmatrix} -0.2\\-0.8\\-0.1\\-0.7\end{bmatrix} \end{bmatrix}$ <br>
 $L_{1\space 2} = L_{2}\cdot L_{1}$
 >$=\begin{bmatrix} 0.5&0.2&-0.1&0.9\\ 0.2&-0.5&0.3&0.1 \end{bmatrix} \cdot \begin{bmatrix} 
0.5&0.2&0&0&-0.2\\
0.2&-0.5&-0.1&0.9&-0.8\\
0&0.2&0&0.1&-0.1\\
0.1&0.8&0.3&0&-0.7\end{bmatrix}$ <br>
$=\begin{bmatrix} 0.5&0.2&-0.1&0.9\\ 0.2&-0.5&0.3&0.1 \end{bmatrix} \cdot \begin{bmatrix} 
 \begin{bmatrix} 0.5\\0.2\\0\\0.1 \end{bmatrix}
 \begin{bmatrix} 0.2\\-0.5\\0.2\\0.8 \end{bmatrix} 
 \begin{bmatrix} 0\\-0.1\\0\\0.3 \end{bmatrix}
 \begin{bmatrix} 0\\0.9\\0.1\\0 \end{bmatrix} 
 \begin{bmatrix} -0.2\\-0.8\\-0.1\\-0.7\end{bmatrix} \end{bmatrix} $<br>
 $=\begin{bmatrix} 
 L_{2} \cdot \begin{bmatrix} 0.5\\0.2\\0\\0.1 \end{bmatrix}& 
 L_{2} \cdot \begin{bmatrix} 0.2\\-0.5\\0.2\\0.8 \end{bmatrix} &
 L_{2} \cdot \begin{bmatrix} 0\\-0.1\\0\\0.3 \end{bmatrix} &
 L_{2} \cdot \begin{bmatrix} 0\\0.9\\0.1\\0 \end{bmatrix} &
 L_{2} \cdot \begin{bmatrix} -0.2\\-0.8\\-0.1\\-0.7\end{bmatrix} \end{bmatrix} $ 
 
 $L_{1\space 2} = \begin{bmatrix} 
 0.38&0.7&0.25&0.17&-0.88\\ 
 0.01&0.43&0.08&-0.42&0.26
 \end{bmatrix}$

In [ ]:
def matrix_matrix_product(matrix1, matrix2):
    # pprint.pprint(matrix1)
    # pprint.pprint(matrix2)
    matrix_product = list()
    for column in np.transpose(matrix2):
        matrix_product.append(matrix_vector_product(column, matrix1))
    matrix_product = np.array(matrix_product)
    return np.transpose(matrix_product)

matrix_l1_l2 = matrix_matrix_product(matrix_layer2, matrix_layer1)
# Multiplied weights:
pprint.pprint(matrix_l1_l2)

 #### Using the new matrix to get the output vector:
 $\vec{y} = L_{1\space 2} \cdot \vec{x}$ <br>
 >$\begin{bmatrix} 
 0.38&0.7&0.25&0.17&-0.88\\ 
 0.01&0.43&0.08&-0.42&0.26
 \end{bmatrix} \cdot \begin{bmatrix} 1\\1\\1\\1\\1 \end{bmatrix}$<br>
 
 $\vec{y} = \begin{bmatrix} 0.62\\0.36 \end{bmatrix}$ <br>
 Just like expected, the resulting vector is the same as that given by method 1. As aforementioned, there is a slight
 discrepancy when calculated by the script.

In [ ]:
# Using the matrix-matrix-product:
pprint.pprint(matrix_vector_product(input_layer, matrix_l1_l2))
